## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Mix Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import seaborn as sns 
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#### <font color='blue'>Libraries for Critical Diagram 

In [7]:
from sklearn.model_selection import StratifiedKFold
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
from Orange.evaluation import compute_CD, graph_ranks
from sklearn.metrics import mean_absolute_error

### <font color='green'> Load Dataset 

In [8]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [9]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [10]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [11]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_std,skinTemperature_var,heartRate_std,heartRate_var,bpCuffMean_std,sao2_std,bpCuffMean_var,...,BIRTH_WEIGHT,bpCuffDiastolic_mean,bpCuffSystolic_mean,temperature_std,temperature_var,sao2_mean,glucometer_mean,bpCuffMean_mean,PLATELET,D10W_SUM
0,0.775318,0.422245,0.178291,0.134078,0.017977,0.22911,0.057937,0.230854,0.097817,0.053294,...,0.264267,0.457143,0.602649,0.125281,0.015695,0.881455,0.312593,0.310696,0.281330,0.156019
1,0.709668,0.583964,0.341014,0.086672,0.007512,0.28258,0.086102,0.142539,0.080938,0.020317,...,0.437819,0.479330,0.644907,0.061399,0.003770,0.926056,0.272551,0.351928,0.392157,0.211970


In [12]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Mix ML) - (+FS, +HO)

In [13]:
def get_individual_result(model, dsel_x, dsel_y, test_x, test_y): 
    model.fit(dsel_x, dsel_y)
    y_preds = model.predict(test_x) 
    yproba = model.predict_proba(test_x)[::,1] 
    
    acc = accuracy_score(test_y, y_preds)
    prec = precision_score(test_y, y_preds)
    rec = recall_score(test_y, y_preds)
    f1 = f1_score(test_y, y_preds)
    fpr, tpr, _ = roc_curve(test_y,  yproba) 
    auc = roc_auc_score(test_y, yproba)
    
    return {"acc": acc, "prec": prec, "rec": rec, "f1":f1, "fpr": fpr, "tpr":tpr, "auc": auc}
    

In [14]:
def hold_out_mix_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
 
    model_lr = LogisticRegression(penalty='l2', C=0.002)
    model_ml_perceptron = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    
    model_rf  = RandomForestClassifier(criterion='gini', n_estimators=100, max_depth=3)
    model_cat = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    
    model_lr.fit(X_train, y_train_en)
    model_ml_perceptron.fit(X_train, y_train_en)
    model_rf.fit(X_train, y_train_en) 
    model_cat.fit(X_train, y_train_en) 
    
    classifiers_names = ["MLP", "Logistic Regression", "Random Forest", 
                         "CatBoost"]
    
    pool_classifiers = [model_ml_perceptron, 
                        model_lr, 
                        model_rf, 
                        model_cat] 
    c_acc_list = [] 
    
    for cls in pool_classifiers:
        y_preds_c = cls.predict(X_test_mm) 
        c_acc_list.append(accuracy_score(y_test, y_preds_c))           
        
        
    classifiers_results =  {'name': classifiers_names, 
                            'accuracy': c_acc_list}
    
    clsDF = pd.DataFrame.from_dict(classifiers_results)
        
    # DES STARTS
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=7) 
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9)
    fire_metades = METADES(pool_classifiers, DFP=True, k=9) 
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=9)
    fire_mcb = MCB(pool_classifiers, DFP=True, k=7) # 7 was 96%
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        results_dct = get_individual_result(e_cls, X_dsel, y_dsel, X_test_mm, y_test)
        acc_list.append(results_dct["acc"])
        precision_list.append(results_dct["prec"])
        recall_list.append(results_dct["rec"]) 
        f1_lists.append(results_dct["f1"]) 
        auc_list.append(results_dct["auc"])
        fpr_list.append(results_dct["fpr"])
        tpr_list.append(results_dct["tpr"])

    
    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df, clsDF

In [15]:
results_data = []
classifier_results_data = []
for i in range(0, 15):
    result, cls_results = hold_out_mix_ML()
    results_data.append(result)
    classifier_results_data.append(cls_results)
    

fireResultsDF = pd.concat(results_data)
classifiersResultsDF = pd.concat(classifier_results_data)

Iteration 1, loss = 0.62349243
Iteration 2, loss = 0.51685240
Iteration 3, loss = 0.43326661
Iteration 4, loss = 0.38106240
Iteration 5, loss = 0.35595421
Iteration 6, loss = 0.33716478
Iteration 7, loss = 0.32348388
Iteration 8, loss = 0.31220436
Iteration 9, loss = 0.29939142
Iteration 10, loss = 0.29059070
Iteration 11, loss = 0.28335706
Iteration 1, loss = 0.61572820
Iteration 2, loss = 0.49469909
Iteration 3, loss = 0.41061859
Iteration 4, loss = 0.36809259
Iteration 5, loss = 0.34161179
Iteration 6, loss = 0.32354979
Iteration 7, loss = 0.30926596
Iteration 8, loss = 0.29741578
Iteration 9, loss = 0.29065757
Iteration 10, loss = 0.28994301
Iteration 11, loss = 0.27849354
Iteration 1, loss = 0.63630806
Iteration 2, loss = 0.53079848
Iteration 3, loss = 0.44755077
Iteration 4, loss = 0.39379647
Iteration 5, loss = 0.36453757
Iteration 6, loss = 0.34400617
Iteration 7, loss = 0.32726466
Iteration 8, loss = 0.31863897
Iteration 9, loss = 0.30885944
Iteration 10, loss = 0.29930432
Ite

Iteration 1, loss = 0.65244638
Iteration 2, loss = 0.54032866
Iteration 3, loss = 0.45300443
Iteration 4, loss = 0.39541743
Iteration 5, loss = 0.36425244
Iteration 6, loss = 0.34076852
Iteration 7, loss = 0.32472320
Iteration 8, loss = 0.31083669
Iteration 9, loss = 0.30436125
Iteration 10, loss = 0.29269398
Iteration 11, loss = 0.28467339
Iteration 1, loss = 0.62319350
Iteration 2, loss = 0.51356192
Iteration 3, loss = 0.43700022
Iteration 4, loss = 0.38962914
Iteration 5, loss = 0.36475365
Iteration 6, loss = 0.34530317
Iteration 7, loss = 0.32730058
Iteration 8, loss = 0.31335156
Iteration 9, loss = 0.30281752
Iteration 10, loss = 0.29549153
Iteration 11, loss = 0.28543523
Iteration 1, loss = 0.63564710
Iteration 2, loss = 0.50897734
Iteration 3, loss = 0.42280079
Iteration 4, loss = 0.37763966
Iteration 5, loss = 0.35633879
Iteration 6, loss = 0.33630293
Iteration 7, loss = 0.32197764
Iteration 8, loss = 0.30949478
Iteration 9, loss = 0.30072093
Iteration 10, loss = 0.29177007
Ite

In [16]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [17]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.933,0.002,0.930,0.005,0.935,0.000,0.933,0.002,0.980,0.003
1,KNORA-U,0.932,0.003,0.929,0.006,0.935,0.000,0.932,0.003,0.980,0.003
2,FIRE-KNORA-E,0.939,0.002,0.942,0.003,0.935,0.000,0.939,0.002,0.985,0.001
3,KNORA-E,0.939,0.002,0.942,0.004,0.935,0.000,0.939,0.002,0.985,0.001
4,FIRE-METADES,0.944,0.008,0.947,0.002,0.940,0.017,0.943,0.009,0.986,0.001
5,METADES,0.944,0.009,0.949,0.003,0.940,0.017,0.944,0.009,0.987,0.001
6,FIRE-DESKNN,0.945,0.002,0.953,0.003,0.935,0.000,0.944,0.002,0.985,0.001
7,DESKNN,0.944,0.002,0.953,0.003,0.935,0.000,0.944,0.001,0.985,0.001
8,FIRE-MCB,0.951,0.005,0.937,0.004,0.967,0.007,0.952,0.005,0.985,0.001
9,MCB,0.952,0.004,0.938,0.005,0.967,0.006,0.953,0.004,0.985,0.002


In [22]:
# finalResultsDF.to_csv("Results/mix_pool_results/des_mix_pool_4_cls_results_2-2.csv", index=False)

In [19]:
classifiers_names = ["MLP", "Logistic Regression", "Random Forest", "CatBoost"] 
# "AdaBoost", "LGBM"

accuracy_list = []
accuracy_std_list = []
for n in classifiers_names:
    accuracy_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.mean())
    accuracy_std_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.std())

final_cls_results = {"classifier": classifiers_names, 
                     "accuracy": accuracy_list, 
                     "accuracy_std": accuracy_std_list}

finalClassifierResultsDF = pd.DataFrame.from_dict(final_cls_results)

In [20]:
finalClassifierResultsDF

,classifier,accuracy,accuracy_std
0,MLP,0.941640,1.744107e-02
1,Logistic Regression,0.943366,4.596760e-16
2,Random Forest,0.940129,1.382675e-03
3,CatBoost,0.940417,7.887381e-03


In [21]:
finalResultsDF.describe().T

,count,mean,std,min,25%,50%,75%,max
accuracy,14.0,0.941357,0.006184,0.932,0.93600,0.9430,0.94400,0.952
accuracy_std,14.0,0.004143,0.003231,0.002,0.00200,0.0025,0.00475,0.012
precision,14.0,0.941286,0.007927,0.929,0.93475,0.9420,0.94700,0.953
precision_std,14.0,0.003857,0.001099,0.002,0.00300,0.0040,0.00475,0.006
recall,14.0,0.940929,0.011391,0.935,0.93500,0.9350,0.94000,0.967
recall_std,14.0,0.005000,0.008048,0.000,0.00000,0.0000,0.00675,0.023
f1_score,14.0,0.941214,0.006351,0.932,0.93600,0.9420,0.94400,0.953
f1_std,14.0,0.004071,0.003430,0.001,0.00200,0.0020,0.00475,0.012
auc,14.0,0.984000,0.002481,0.979,0.98400,0.9850,0.98500,0.987
auc_std,14.0,0.001929,0.001900,0.001,0.00100,0.0010,0.00200,0.008
